In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
path = 'file:///D:/Documents/Datasets/train.csv'
ps = 'file:///D:/Documents/Datasets/test.csv'
train = pd.read_csv(path)
test = pd.read_csv(ps)
train_data = train.copy()
test_data = test.copy()

vectors = test_data.columns

train_data = train_data.loc[:, vectors]
test_data = test_data.loc[:, vectors]
drop_list = []

for i in test_data.columns:
    if test_data[i].isna().sum() >= test_data[i].notnull().sum():
        drop_list.append(i)

for i in train_data.columns:
    if train_data[i].isna().sum() >= train_data[i].notnull().sum():
        drop_list.append(i)
        
print(drop_list)

drop_list = list(set(drop_list))
train_data.drop(drop_list, axis=1, inplace=True)
test_data.drop(drop_list, axis=1, inplace=True)

for i in test_data.columns:
    nulls_value = test_data[i].isna().sum()
    message = "Column {} has {} nulls".format(i, nulls_value)

aux = train.copy()
aux2 = train.copy()

aux = aux.loc[:, test_data.columns]
aux['NU_NOTA_MT'] = aux2.NU_NOTA_MT

c = aux.corr()
c.NU_NOTA_MT.sort_values()

new_vector_training = [
    'NU_NOTA_COMP1',
    'NU_NOTA_COMP2',
    'NU_NOTA_COMP4',
    'NU_NOTA_COMP5',
    'NU_NOTA_COMP3',
    'NU_NOTA_REDACAO',
    'NU_NOTA_LC',
    'NU_NOTA_CH',
    'NU_NOTA_CN',
    'NU_NOTA_MT'
]

new_vector_test = [
    'NU_INSCRICAO',
    'NU_NOTA_COMP1',
    'NU_NOTA_COMP2',
    'NU_NOTA_COMP4',
    'NU_NOTA_COMP5',
    'NU_NOTA_COMP3',
    'NU_NOTA_REDACAO',
    'NU_NOTA_LC',
    'NU_NOTA_CH',
    'NU_NOTA_CN'
]

train_data = train.copy()
train_data = train_data.loc[:, new_vector_training]
train_data.dropna(subset=['NU_NOTA_MT'], inplace=True)
y = train_data.NU_NOTA_MT
X = train_data.drop(['NU_NOTA_MT'], axis=1)

validation_data = test.copy()
validation_data_1 = validation_data.loc[:, new_vector_test]
validation_data_2 = validation_data.loc[:, new_vector_test]

train_X, validation_X, train_y, validation_y = train_test_split(X, y, random_state = 0)

model = XGBRegressor(n_estimators=200, learning_rate=0.1)
model.fit(train_X, train_y, early_stopping_rounds=5, eval_set=[(validation_X, validation_y)], verbose=False)

validation_data_1.drop(['NU_INSCRICAO'], axis=1, inplace=True)
predicted_nota = model.predict(validation_data_1)
answer_df = pd.DataFrame({'NU_INSCRICAO': validation_data_2.NU_INSCRICAO, 'NU_NOTA_LC': validation_data_2.NU_NOTA_LC, 'NU_NOTA_MT_PREDICT': predicted_nota})
def replace_notes(row):
    if row.NU_NOTA_LC == np.NaN:
        return np.NaN
    return row.NU_NOTA_MT_PREDICT

answer_df['NU_NOTA_MT'] = answer_df.apply(replace_notes, axis='columns')
answer_df.loc[answer_df.NU_NOTA_LC.isna(), ['NU_NOTA_MT']] = np.NaN
answer_df_final = answer_df.loc[:, ['NU_INSCRICAO', 'NU_NOTA_MT']]
answer_df_final.to_csv('answer.csv', index=False)

['TP_ENSINO', 'TP_DEPENDENCIA_ADM_ESC', 'Q027', 'TP_ENSINO', 'TP_DEPENDENCIA_ADM_ESC', 'Q027']
